Pip Install

In [ ]:
#pip install requests

Imports

In [21]:
import requests
import csv
import pandas as pd

Searching for projects that contains .dbml files.

In [ ]:

def filter_dbml_files(results):
    filtered_results = [code_file for code_file in results if code_file['name'].endswith(".dbml")]
    return filtered_results

def search_code_with_dbml_files(access_token, page=1):
    base_url = 'https://api.github.com/search/code'
    headers = {
        'Authorization': f'token {access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    params = {
        'q': f'filename:.dbml',
        'per_page': 100,  # Adjust the number of results per page as needed
        'page': page  # Specify the current page number
    }

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        files = data.get('items', [])
        return files
    else:
        print(f"Error: {response.status_code}")
        return []

# Replace 'YOUR_ACCESS_TOKEN' with your actual GitHub access token
access_token = 'YOUR_ACCESS_TOKEN'
results = []

# Perform multiple requests to gather all results
visited_repositories = set()  # Store visited repositories

# Perform multiple requests to gather all results
for page_number in range(1, 4):  # Adjust the range based on the number of desired pages
    page_results = search_code_with_dbml_files(access_token, page=page_number)
    
    for code_file in page_results:
        repository_name = code_file['repository']['name']
        if repository_name not in visited_repositories:
            results.append(code_file)
            visited_repositories.add(repository_name)


print(f"Total number of results: {len(results)}")

filtered_results = filter_dbml_files(results)
print(f"Total number of filtered results: {len(filtered_results)}")


# Save the repository URL and file URL in a CSV file
csv_filename = "filtered_files.csv"
with open(csv_filename, mode='w', newline='') as file:
    fieldnames = ['Repository', 'Repository URL', 'File', 'File URL']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for code_file in filtered_results:
        writer.writerow({
            'Repository': code_file['repository']['name'],
            'Repository URL': code_file['repository']['html_url'],
            'File': code_file['name'],
            'File URL': code_file['html_url'],
        })

print(f"Filtered files have been saved to {csv_filename}.")

Finding the primary coding langauge for each project

In [ ]:
repositories_csv = "PATH_TO_csv_FILE/filtered_files.csv"
df = pd.read_csv(repositories_csv)
repository_urls = df['Repository URL'].tolist()


# Your GitHub personal access token
access_token = "YOUR_ACCESS_TOKEN"

# Headers with authentication
headers = {
    "Authorization": f"token {access_token}"
}

data_list = []

# Iterate through the repository URLs
for repository_url in repository_urls:
    # Parse repository owner and name from the URL
    owner, repo_name = repository_url.split("/")[-2:]

    # GitHub API endpoint for repository information
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}"

    # Make a request to the GitHub API with authentication headers
    response = requests.get(api_url, headers=headers)
    data = response.json()

    # Extract the primary programming language from the API response
    primary_language = data.get("language")

    # Append data to the list
    data_list.append({"Repository URL": repository_url, "Primary Language": primary_language})

# Create a DataFrame from the list
df_results = pd.DataFrame(data_list)

# Merge the results with the original df_testing based on the "Repository URL" column
df = pd.merge(df, df_results, on="Repository URL", how="left")

# Display the updated DataFrame
csv_filename = "output.csv"

# Save the DataFrame to the CSV file
df.to_csv(csv_filename, index=False)  # Set index=False to exclude the index column from the CSV

print(f"DataFrame has been saved to {csv_filename}.")

print(df.head())
